In [1]:
'''
验证光大证券报告中的数据
'''
import pandas
import os
from datetime import datetime
import numpy

current_path = os.path.abspath('')
df = pandas.read_excel(current_path + '/../data/data.xlsx')
df['上市公告日期'] = pandas.to_datetime(df['上市公告日期'],format="%Y-%m-%d")
# 2019年至2022年11月18日 共558只可转债
start = pandas.Timestamp("2019-01-01")
end = pandas.Timestamp("2022-11-18")
df = df[(df['上市公告日期']>=start) & (df['上市公告日期']<=end)]
count = df['上市公告日期'].count()
print("可转债数量: {}".format(count))


可转债数量: 558


In [2]:
# 转债发行及上市首日表现情况
year = {
    '整体' : [pandas.Timestamp("2019-01-01"), pandas.Timestamp("2022-11-18")],
    '2019' : [pandas.Timestamp("2019-01-01"), pandas.Timestamp("2019-12-31")],
    '2020' : [pandas.Timestamp("2020-01-01"), pandas.Timestamp("2020-12-31")],
    '2021' : [pandas.Timestamp("2021-01-01"), pandas.Timestamp("2021-12-31")],
    '2022' : [pandas.Timestamp("2022-01-01"), pandas.Timestamp("2022-12-31")]
}
for d, v in year.items():
    count = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['上市公告日期'].count()
    total = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['发行金额(万元)'].sum()
    mean = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['上市首日收盘价'].apply(lambda x : (x-100)/100).mean()
    median = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['上市首日收盘价'].apply(lambda x : (x-100)/100).median()
    print("{}\t 发行规模(亿元): {:.2f}\t 发行只数: {}\t 涨跌幅平均数: {:.2%}\t 涨跌幅中位数: {:.2%}".format(d, total/10000, count, mean, median))


整体	 发行规模(亿元): 10048.21	 发行只数: 558	 涨跌幅平均数: 19.82%	 涨跌幅中位数: 17.42%
2019	 发行规模(亿元): 2477.81	 发行只数: 106	 涨跌幅平均数: 9.44%	 涨跌幅中位数: 8.77%
2020	 发行规模(亿元): 2466.81	 发行只数: 204	 涨跌幅平均数: 19.10%	 涨跌幅中位数: 17.19%
2021	 发行规模(亿元): 2697.81	 发行只数: 121	 涨跌幅平均数: 19.27%	 涨跌幅中位数: 18.48%
2022	 发行规模(亿元): 2405.78	 发行只数: 127	 涨跌幅平均数: 30.18%	 涨跌幅中位数: 25.90%


In [3]:
# 股东配售比例
df['股东配售比例'] = df.apply(lambda x : x['向原股东配售数量（张）']/(x['发行金额(万元)']*10000/100), axis=1)
for d, v in year.items():
    ratio = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['股东配售比例'].mean()
    ratio2 = numpy.mean([ float(x.split('%')[0]) for x in df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['股东配售率'] ])
    print("{}\t 股东平均配售比例: {:.2%}\t {:.2f}%".format(d, ratio, ratio2))

整体	 股东平均配售比例: 60.27%	 60.31%
2019	 股东平均配售比例: 43.68%	 44.00%
2020	 股东平均配售比例: 64.17%	 64.17%
2021	 股东平均配售比例: 59.58%	 59.58%
2022	 股东平均配售比例: 68.52%	 68.40%


In [4]:
t = {
    'T-2' : 'T-2日',
    'T-1' : 'T-1日', 
}
# 从平均数看， 如在T-2日参与转债抢权，则一手转债成本1000元，平均可盈利93.6元，其中可转债收益201.2元，股票亏损107.5元
df['可转债上市当天成交均价'] = df.apply(lambda x : x['上市首日成交金额(万)']/(x['发行金额(万元)']*10*x['上市首日换手率(%)']) * 100 * 1000, axis=1)
for key, value in t.items():
    print(key)
    for d, v in year.items():
        total = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['可转债上市当天成交均价'].apply(lambda x : x-100).sum()
        count = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['可转债上市当天成交均价'].count()
        profit_cb = total / count * 10 # 每手可转债10股，乘以10
        total = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])].apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x[value + '均价']), axis=1).sum()
        profit_st = total / count
        profit_avg = profit_cb + profit_st
        money_st = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])].apply(lambda x : 1000 / x['每股配售（元）'] * x[value + '均价'], axis=1).sum() / count
        print("{}\t 每手正股参与金额: {:.2f}\t 每手可转债净收益: {:.2f}\t 转债收益: {:.2f}\t 股票收益: {:.2f}".format(d, money_st, profit_avg, profit_cb, profit_st))


T-2
整体	 每手正股参与金额: 14898.16	 每手可转债净收益: 93.64	 转债收益: 201.15	 股票收益: -107.52
2019	 每手正股参与金额: 12428.11	 每手可转债净收益: 170.01	 转债收益: 93.48	 股票收益: 76.52
2020	 每手正股参与金额: 15345.84	 每手可转债净收益: 96.93	 转债收益: 200.31	 股票收益: -103.38
2021	 每手正股参与金额: 16870.29	 每手可转债净收益: 74.58	 转债收益: 192.88	 股票收益: -118.30
2022	 每手正股参与金额: 14361.73	 每手可转债净收益: 42.76	 转债收益: 300.26	 股票收益: -257.50
T-1
整体	 每手正股参与金额: 14943.98	 每手可转债净收益: 47.82	 转债收益: 201.15	 股票收益: -153.33
2019	 每手正股参与金额: 12477.50	 每手可转债净收益: 120.62	 转债收益: 93.48	 股票收益: 27.13
2020	 每手正股参与金额: 15410.01	 每手可转债净收益: 32.76	 转债收益: 200.31	 股票收益: -167.55
2021	 每手正股参与金额: 16881.11	 每手可转债净收益: 63.75	 转债收益: 192.88	 股票收益: -129.13
2022	 每手正股参与金额: 14408.42	 每手可转债净收益: -3.94	 转债收益: 300.26	 股票收益: -304.20


In [5]:
# 转换价值和转股溢价率
df['转股价值'] = df.apply(lambda x : 100/x['初始转股价'] * x['可转债上市日正股收盘价'], axis=1)
for d, v in year.items():
    price_avg = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['上市首日收盘价'].mean()
    count = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['上市首日收盘价'].count()
    convertion_price = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]['转股价值'].sum() / count
    premium_ratio = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])].apply(lambda x : (x['上市首日收盘价']-x['转股价值'])/x['转股价值'], axis=1).mean() * 100
    print("{}\t 可转债上市首日 平均价格: {:.2f}\t 平均转股价值: {:.2f}\t 平均转股溢价率: {:.2f}%".format(d, price_avg, convertion_price, premium_ratio))


整体	 可转债上市首日 平均价格: 119.82	 平均转股价值: 100.36	 平均转股溢价率: 21.19%
2019	 可转债上市首日 平均价格: 109.44	 平均转股价值: 101.18	 平均转股溢价率: 8.93%
2020	 可转债上市首日 平均价格: 119.10	 平均转股价值: 103.37	 平均转股溢价率: 16.56%
2021	 可转债上市首日 平均价格: 119.27	 平均转股价值: 100.20	 平均转股溢价率: 19.97%
2022	 可转债上市首日 平均价格: 130.18	 平均转股价值: 95.01	 平均转股溢价率: 40.04%


In [6]:
# 股价波动
t = {
    'T-3' : 'T-3日',
    'T-2' : 'T-2日',
    'T-1' : 'T-1日', 
    'T' : 'T日'
}
print("平均数\t T-3\t T-2\t T-1\t T")
for d, v in year.items():
    limit_avg = [ '{:.2f}%'.format(df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])][value + '涨跌幅'].mean()) for key, value in t.items() ]
    print("{}\t {}".format(d, ' \t'.join(limit_avg)))

平均数	 T-3	 T-2	 T-1	 T
整体	 0.53% 	1.85% 	0.24% 	-1.60%
2019	 0.56% 	1.40% 	0.50% 	-0.23%
2020	 0.49% 	1.97% 	0.37% 	-1.72%
2021	 0.50% 	2.13% 	-0.14% 	-1.20%
2022	 0.58% 	1.78% 	0.19% 	-2.95%
